In [1]:
import sys
import os
import pandas as pd
import numpy as np
from datetime import datetime
import pymysql



from dotenv import load_dotenv
load_dotenv()

True

In [2]:
HOSTCaris =os.getenv("HOSTCaris")
DBCaris =os.getenv("DBCaris")
USRCaris =os.getenv("USRCaris")
PASSCaris =os.getenv("PASSCaris")

connection = pymysql.connect(
    host=HOSTCaris,
    user=USRCaris,
    password=PASSCaris,
    db=DBCaris
)

In [3]:
req='''SELECT *
FROM
    caris_db.view_muso_household_report
where (isnull(indice_householdcount) and (is_household_applicable ='no' or isnull(is_household_applicable)))'''

In [4]:
compt=pd.read_sql(req,connection)

/usr/local/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [5]:
compt

,indice_householdcount,id_patient,prenom,nom,id_group,group_name,office,commune,code,case_id_groupes,groups_closed_on_commcare,is_household_applicable,is_inactive_groupes,is_graduated_groupes,is_inactive_members,is_graduated_members,members_closed_on_commcare
0,None,32730,Benitha,Telfort,562,Tet Ansan de Lahatte Dubedou,GON,Gonaïves,153,a06e281f3ad543fb989b1821248f209b,0,no,0.0,0.0,0,0.0,0
1,None,56769,Thelusma,Francine,276,Espwa pou demen,CAP,Capotille,63,2d793a0837ca4474be22d1c0aa382224,0,no,0.0,0.0,0,0.0,0
2,None,66696,Guerline,Sidnat,766,Leve Kanpe,GON,Gonaïves,261,77d7c30ec238446ba4ead87ba43e6494,0,no,0.0,0.0,0,NaN,0
3,None,74488,Marie,Petit-Frere,967,Edem avanse plis,GON,Gonaïves,281,6a120fff65a6441d9f707011c774c934,0,no,0.0,0.0,0,NaN,0
4,None,97846,Yphilia,Olizaire,1216,Christ Capable,CAY,Les Cayes,218,5047a58ac2874bc0bf857a8ed8452ba9,0,no,0.0,0.0,0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3261,None,169130,Edwine,jean Pierre,1328,OCUDN 2,PAP,Pétionville,266,e8ce8a05c70449a6a29c4bddcf26adee,0,None,0.0,0.0,0,0.0,0
3262,None,169131,Raymond,Islande,1824,Fanm solid,CAP,Trou-du-Nord,472,ee8ffa0e-466b-4ab8-909c-e49e15c8bcdd,0,None,0.0,0.0,0,0.0,0
3263,None,169132,Imacule,Sylvestre,1824,Fanm solid,CAP,Trou-du-Nord,472,ee8ffa0e-466b-4ab8-909c-e49e15c8bcdd,0,None,0.0,0.0,0,0.0,0
3264,None,169133,Rose-Carmelle,Prophète,1824,Fanm solid,CAP,Trou-du-Nord,472,ee8ffa0e-466b-4ab8-909c-e49e15c8bcdd,0,None,0.0,0.0,0,0.0,0


In [6]:
compt.to_excel('cnf_muso.xlsx',index=False,na_rep="")

In [7]:
resume = compt.pivot_table(
    values="id_patient",
    index="office",
    aggfunc='count',
    fill_value=0,
    margins=True,
    margins_name="Total"
)
resume

,id_patient
office,
BOM,4
CAP,1396
CAY,354
GON,1189
JER,60
PAP,235
PDP,28
Total,3266


In [8]:
off=compt["office"].unique()
off=off.tolist()

In [9]:
writer = pd.ExcelWriter(f"Comptage non faits_{str(datetime.today().strftime('%Y-%m-%d'))}.xlsx", engine='xlsxwriter')
resume.to_excel(writer,index=True,sheet_name="resume")
for name in off:
    cond=compt["office"]== name
    file=compt[cond]
    file.to_excel(writer, index=False, sheet_name=f'{name}')
writer.save()